In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.relative_locator import locate_with

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Output dataframe
df = pd.DataFrame({'Name:':[],
                    'Phone:':[],
                    'e-mail:':[],
                    'Web:':[],
                    'Manager(s):':[],
                    'Other Pers.:':[],
                    'Attractions:':[],
                    'Aud. Cap.:':[],
                    'Artist Budget:':[],
                    'Events:':[]})

In [6]:
# Access website
driver = webdriver.Chrome()
driver.get("https://www.musicalamerica.com/datalistings/index.cfm")
driver.minimize_window()

In [7]:
# Select "US/Canada Performing Arts Series" option
driver.find_element(By.ID, 't3').click()
driver.find_element(By.XPATH , '/html/body/div[6]/div/div[1]/div/div/div[4]/div/div[2]/div[1]/div[2]/input[5]').click()
driver.find_element(By.XPATH , '/html/body/div[6]/div/div[1]/div/div/div[4]/div/div[2]/div[7]/div/form/div[9]/input[1]').click()

In [19]:
# Subscriber login
username = input("Username: ")
password = input("Password: ")
driver.find_element(By.NAME, 'username').send_keys(username)
driver.find_element(By.NAME, 'password').send_keys(password)
driver.find_element(locate_with(By.NAME, 'submit').below({By.NAME: 'password'})).click()

In [ ]:
# Show the number of musicals found and the number of pages that will be extracted 
qty_result = int(driver.find_element(By.XPATH, '/html/body/div[6]/div/div[1]/div/div/div[4]/div/div[3]/div[2]').text.split(" ")[-1])
total_pages = qty_result // 10 + 1
print(qty_result, total_pages)

In [ ]:
# Extract information from the first to the penultimate page
for p in range(total_pages - 1):
    if p == 0:
        xpath_next = "/html/body/div[6]/div/div[1]/div/div/div[4]/div/div[3]/div[4]/form/input[14]"
    else:
        xpath_next = "/html/body/div[6]/div/div[1]/div/div/div[4]/div/div[3]/div[4]/form/input[15]"
    WebDriverWait(driver, timeout=10).until(lambda x: x.find_element(By.XPATH, xpath_next))
    result = driver.find_element(By.ID, 'usperf_results')
    row_dict = {'Name:': '',
        'Phone:': '',
        'e-mail:': '',
        'Web:': '',
        'Manager(s):': '',
        'Other Pers.:': '',
        'Attractions:': '',
        'Aud. Cap.:': '',
        'Artist Budget:': '',
        'Events:': ''}
    titles = result.find_elements(By.TAG_NAME, 'strong')
    for title in titles:
        row_dict['Name:'] = title.text
        parent = title.find_elements(By.XPATH, './../..')
        datarow = parent[0].find_elements(By.ID, 'datarow')
        for data in datarow:
            indice = data.find_element(By.ID, 'leftlabel2').text
            valor = data.find_element(By.ID, 'rightdata').text.replace('\n', ', ')
            row_dict[indice] = valor
        df = df.append(row_dict, ignore_index=True, sort=False)
    driver.find_element(By.XPATH, xpath_next).click()

In [ ]:
# Extract information from de last page
result = driver.find_element(By.ID, 'usperf_results')
row_dict = {'Name:': '',
    'Phone:': '',
    'e-mail:': '',
    'Web:': '',
    'Manager(s):': '',
    'Other Pers.:': '',
    'Attractions:': '',
    'Aud. Cap.:': '',
    'Artist Budget:': '',
    'Events:': ''}
titles = result.find_elements(By.TAG_NAME, 'strong')
for title in titles:
    row_dict['Name:'] = title.text
    parent = title.find_elements(By.XPATH, './../..')
    datarow = parent[0].find_elements(By.ID, 'datarow')
    for data in datarow:
        indice = data.find_element(By.ID, 'leftlabel2').text
        valor = data.find_element(By.ID, 'rightdata').text.replace('\n', ', ')
        row_dict[indice] = valor
    df = df.append(row_dict, ignore_index=True, sort=False)
driver.close()

In [ ]:
# Show result
df

In [ ]:
# Export result
df.to_excel('performing_arts_series.xlsx')